In [2]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.7896915 ,  0.08202791,  0.02420156, -0.10216922, -0.06612323,
         0.04063302,  0.03854873, -0.17339855,  0.16333175,  0.19726057]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.04670663, 0.11167642, 0.10540174, 0.0928893 , 0.09629866,
        0.10714795, 0.10692486, 0.08650301, 0.12113545, 0.12531595]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.2335448

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2556 - accuracy: 0.9238
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1104 - accuracy: 0.9662
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0783 - accuracy: 0.9761
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0611 - accuracy: 0.9809
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0505 - accuracy: 0.9841


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0913 - accuracy: 0.9722 - 668ms/epoch - 2ms/step


[0.09126579761505127, 0.9721999764442444]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.0216050e-08, 1.2223073e-07, 3.2204589e-05, 1.7835568e-04,
        7.6039464e-10, 6.3247711e-08, 7.6269321e-12, 9.9978572e-01,
        2.3711877e-08, 3.4841642e-06],
       [1.5697229e-12, 7.0097111e-04, 9.9929869e-01, 3.7174269e-07,
        7.5264170e-12, 1.4762674e-08, 3.4855252e-09, 1.9372062e-09,
        6.3101222e-09, 5.5018785e-12],
       [9.5293773e-09, 9.9981970e-01, 1.0131909e-05, 7.9165012e-08,
        8.0636273e-05, 6.0351857e-07, 5.2338884e-05, 2.8695347e-05,
        7.3856095e-06, 5.3777097e-07],
       [9.6829826e-01, 4.6502453e-05, 2.1681510e-02, 7.8707031e-04,
        7.6908094e-05, 7.6385681e-05, 2.8059576e-03, 1.7969955e-05,
        1.5648127e-04, 6.0530240e-03],
       [1.0113722e-07, 1.2317339e-07, 1.4147026e-05, 2.7693444e-08,
        9.9848193e-01, 4.5350592e-07, 1.4341906e-07, 1.8555189e-05,
        3.0232876e-08, 1.4845064e-03]], dtype=float32)>